In [2]:
#!/usr/bin/env python3
import os
import shutil
import pandas as pd
import math
from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment
from openpyxl.utils import get_column_letter

# === Einstellungen ===
FONT = Font(name='Arial', size=8)
FONT_BOLD = Font(name='Arial', size=8, bold=True)
thin = Side(border_style='thin', color='000000')
border = Border(left=thin, right=thin, top=thin, bottom=thin)

# Globale Anforderungen
REQUIRED = [
    'model', 'algorithmus', 'typ', 'keygen_time', 'enc_time', 'dec_time',
    'keygen_heap', 'enc_heap', 'dec_heap',
    'keygen_stack', 'enc_stack', 'dec_stack'
]

# Funktion zum Formatieren mit signifikanter Genauigkeit
def format_sig(x, sig=4):
    if pd.isna(x):
        return ''
    if x == 0:
        return '0'
    order = int(math.floor(math.log10(abs(x))))
    decimals = sig - 1 - order
    if decimals < 0:
        decimals = 0
    s = f"{x:.{decimals}f}"
    return s.replace('.', ',')

MODEL_ORDER = ['Pi4b', 'Pi3b', 'PiZero2', 'PiBanana', 'PiZero', 'Pi1bPlus', 'Pi1b']

# === Pfade ===
csv_path = 'output/csv/liboqs_all.csv'
out_dir   = 'output/xlsx/liboqs_scaled'

# Ausgabeverzeichnis neu anlegen
def prepare_output(dir_path):
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.makedirs(dir_path, exist_ok=True)

# Maßstabsbestimmung
def determine_scale(values, is_bytes=False):
    arr = values.flatten()
    nz = arr[arr > 0]
    m = nz.min() if len(nz) else 0
    if is_bytes:
        if m >= 2**20: return 2**20, 'MiB'
        if m >= 2**10: return 2**10, 'KiB'
        return 1, 'B'
    else:
        if m >= 1e6: return 1e6, 's'
        if m >= 1e3: return 1e3, 'ms'
        return 1, 'µs'

# Spalten formattieren
def format_columns(df, cols, scale):
    if scale == 1:
        for c in cols:
            df[c] = df[c].apply(lambda v: '' if pd.isna(v) else str(int(v)))
    else:
        for c in cols:
            df[c] = df[c].apply(lambda v: format_sig(v/scale))

# Kopfzeilen schreiben
def write_headers(ws, titles, headers):
    # Model-Spalte bereits gemerged
    for (title, start, end) in zip(titles, [2,5,8], [4,7,10]):
        ws.merge_cells(start_row=1, start_column=start, end_row=1, end_column=end)
        for col in range(start, end+1):
            cell = ws.cell(row=1, column=col)
            cell.border = border
            if col == start:
                cell.value = title
                cell.font = FONT_BOLD
                cell.alignment = Alignment(horizontal='center', vertical='center')
    for idx, h in enumerate(headers, start=1):
        if idx == 1:
            continue
        cell = ws.cell(row=2, column=idx, value=h)
        cell.font = FONT_BOLD
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = border

# Daten schreiben
def write_data(ws, df):
    for r_idx, row in enumerate(df.itertuples(index=False), start=3):
        for c_idx, val in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=val)
            cell.font = FONT
            cell.alignment = Alignment(horizontal='right', vertical='center')
            cell.border = border

# Einzeldatei erstellen
def save_one(algo, grp):
    safe = algo.replace('/', '_').replace(' ', '_')
    wb = Workbook()
    ws = wb.active
    ws.title = 'liboqs All'

    # Model-Zelle merge
    ws.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)
    cell = ws.cell(row=1, column=1, value='Model')
    cell.font = FONT_BOLD
    cell.alignment = Alignment(horizontal='center', vertical='center')
    ws.cell(row=1, column=1).border = border
    ws.cell(row=2, column=1).border = border

    # Typ auslesen
    typ = grp.iloc[0]['typ'].strip().lower()
    if typ == 'sig':
        colnames = ['Key', 'Sign', 'Verify']
    else:
        colnames = ['Key', 'Enc', 'Dec']

    # DataFrame vorbereiten und sortieren
    temp = grp.copy()
    temp = temp.drop(columns=['algorithmus', 'typ'])[REQUIRED[0:1] + REQUIRED[3:]]
    temp['model'] = pd.Categorical(temp['model'], categories=MODEL_ORDER, ordered=True)
    temp = temp.sort_values('model')

    # Skalierungen
    t_scale, t_unit = determine_scale(temp[['keygen_time','enc_time','dec_time']].values)
    h_scale, h_unit = determine_scale(temp[['keygen_heap','enc_heap','dec_heap']].values, is_bytes=True)
    s_scale, s_unit = determine_scale(temp[['keygen_stack','enc_stack','dec_stack']].values, is_bytes=True)

    # Werte formatieren
    format_columns(temp, ['keygen_time','enc_time','dec_time'], t_scale)
    format_columns(temp, ['keygen_heap','enc_heap','dec_heap'], h_scale)
    format_columns(temp, ['keygen_stack','enc_stack','dec_stack'], s_scale)

    # Header
    titles  = [f'Time ({t_unit})', f'Heap ({h_unit})', f'Stack ({s_unit})']
    headers = ['Model'] + colnames * 3
    temp.columns = headers

    # Kopf schreiben
    write_headers(ws, titles, headers)
    write_data(ws, temp)

    # Spaltenbreiten & Zeilenhöhen
    for i in range(1, len(headers)+1):
        col = get_column_letter(i)
        max_len = max(len(str(ws.cell(row=r, column=i).value or '')) for r in range(2, ws.max_row+1))
        ws.column_dimensions[col].width = max_len + 1
    for r in range(1, ws.max_row+1):
        ws.row_dimensions[r].height = 12

    out_path = os.path.join(out_dir, f'{safe}-liboqs.xlsx')
    wb.save(out_path)
    print(f"Erstellt: {out_path}")

# Hauptfunktion
def main():
    prepare_output(out_dir)
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip().str.lower()
    df = df.rename(columns={
        'encaps_time': 'enc_time',
        'decaps_time': 'dec_time',
        'heap_key':    'keygen_heap',
        'heap_enc':    'enc_heap',
        'heap_dec':    'dec_heap',
        'stack_key':   'keygen_stack',
        'stack_enc':   'enc_stack',
        'stack_dec':   'dec_stack',
    })
    miss = [c for c in REQUIRED if c not in df.columns]
    if miss:
        raise RuntimeError(f"Fehlende Spalten: {miss}")
    for algo, grp in df.groupby('algorithmus'):
        save_one(algo, grp)

if __name__ == '__main__':
    main()

Erstellt: output/xlsx/liboqs_scaled\BIKE-L1-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\BIKE-L3-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\BIKE-L5-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-348864-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-348864f-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-460896-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-460896f-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-6688128-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-6688128f-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-6960119-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-6960119f-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-8192128-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Classic-McEliece-8192128f-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Dilithium2-liboqs.xlsx
Erstellt: output/xlsx/liboqs_scaled\Dilith